In [1]:
%run ~/af2genomics/notebooks/__init__.ipynb

prody not found; if needed, install with: conda install conda-forge::prody


In [2]:
df_ = pd.read_csv('UP000005640_9606_HUMAN_v4.txt', names=['pdb'])
df_['af2_id'] = df_['pdb'].map(os.path.basename).str.removesuffix('-model_v4.pdb.gz').str.removeprefix('AF-')
df_['frag_id'] = df_['af2_id'].map(lambda af2_id: af2_id.split('-')[-1])
df_['uniprot_id'] = df_['af2_id'].map(lambda af2_id: (af2_id[::-1].split('-', maxsplit=1)[1])[::-1])
df_['n_frags'] = df_.groupby(['uniprot_id'])['af2_id'].transform('size')
df_ = df_[['uniprot_id', 'af2_id', 'frag_id', 'n_frags', 'pdb']]
df_['af2_pdb'] = df_['af2_id'].map(lambda af2_id: pfile(struct_id=af2_id, step='af2', suffix='.pdb', base=''))
#df_ = df_.head(1000)

In [3]:
def get_resid_pLDDT(fp_):
    resseq_pLDDT = collections.OrderedDict()
    parser = Bio.PDB.PDBParser(QUIET=True)
    structure = parser.get_structure(fp_, fp_)
    for chains in structure:
        for chain in chains:
            for residue in chain:
                resname = residue.get_resname()
                hetflag, resseq, icode = residue.get_id()
                for atom in residue:
                    resseq_pLDDT[resseq] = atom.bfactor
    return resseq_pLDDT

def get_mean_pLDDT(fp_):
    return np.mean(list(pLDDT for resseq, pLDDT in get_resid_pLDDT(fp_).items()))

df_['mean_pLDDT'] = tqdm.contrib.concurrent.process_map(get_mean_pLDDT, df_['af2_pdb'], max_workers=int(os.environ['SLURM_NTASKS']), chunksize=10)

  0%|          | 0/23391 [00:00<?, ?it/s]

In [4]:
def seq_(fp_):
    parser = Bio.PDB.PDBParser(QUIET=True)
    structure = parser.get_structure(fp_, fp_)
    return str(Bio.PDB.Polypeptide.PPBuilder().build_peptides(structure)[0].get_sequence())

#df_['af2_seq'] = df_['af2_pdb'].map(seq_)
df_['seq'] = tqdm.contrib.concurrent.process_map(seq_, df_['af2_pdb'], max_workers=int(os.environ['SLURM_NTASKS']), chunksize=10)

  0%|          | 0/23391 [00:00<?, ?it/s]

In [5]:
with open('af2.fasta', 'w') as fh:
    for i, r in df_.iterrows():
        print(f'>{r.af2_id}', file=fh)
        print(r.seq, file=fh)

In [6]:
!wc -l af2.fasta
!md5sum af2.fasta

46782 af2.fasta
3169677ebd94e4555ad161ae59f8a274  af2.fasta


In [7]:
df_.to_csv('af2.tsv', sep='\t', header=True, index=False, float_format='%.2f')
!wc -l af2.tsv
!md5sum af2.tsv

23392 af2.tsv
e64a2cf56d207b23a6bdccb8f8423e78  af2.tsv
